# NHL Draft dataset
# Exploratory data analysis
# Records: 
## Most Goals by a Rookie in a Single Season
This notebook presents Exploratory Data Analysis performed on NHL Records data obtained from NHL Records API Records endpoint.
### Data collection summary
Dataset generated from a JSON received from the NHL Records API, contains response to the request for all draft records.

For details, see notebook `notebooks/feature_extraction/nhl_api.ipynb`.
